# Train IMDb Classifier
> Train a IMDb classifier with DistilBERT.

In [ ]:
!huggingface-cli login

## Load IMDb dataset

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
ds = load_dataset("imdb")

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
ds['train'].features

{'label': ClassLabel(num_classes=2, names=['neg', 'pos'], names_file=None, id=None),
 'text': Value(dtype='string', id=None)}

## Load Pretrained DistilBERT

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Prepocess Data

In [ ]:
def tokenize(examples):
    outputs = tokenizer(examples['text'], truncation=True)
    return outputs

tokenized_ds = ds.map(tokenize, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

## Prepare Trainer

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

In [ ]:
import numpy as np

def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(num_train_epochs=1,
                                  output_dir="distilbert-imdb",
                                  push_to_hub=True,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  evaluation_strategy="epoch")

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)

In [ ]:
trainer = Trainer(model=model, tokenizer=tokenizer,
                  data_collator=data_collator,
                  args=training_args,
                  train_dataset=tokenized_ds["train"],
                  eval_dataset=tokenized_ds["test"], 
                  compute_metrics=compute_metrics)

## Train Model and Push to Hub

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub()